# Análisis Didáctico de Chi-Cuadrado: Rendimiento vs. Abandono

Este notebook tiene como objetivo explicar paso a paso cómo funciona la prueba de Chi-Cuadrado ($\chi^2$) utilizando tus propios datos. 

Vamos a desglosar:
1.  **La Tabla de Contingencia**: Qué observamos.
2.  **La Matemática**: Cómo calculamos lo que "deberíamos" ver si no hubiera relación.
3.  **Las Asunciones**: ¿Es verdad que necesitamos normalidad? (Spoiler: No).
4.  **El Resultado**: Qué significa ese valor $p$.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configuración visual
sns.set_theme(style="whitegrid")

# Cargar datos (ajustando la ruta relativa asumiendo que el notebook está en 'src')
try:
    df = pd.read_csv("../data/processed/datos_limpios.csv")
    print("¡Datos cargados correctamente!")
except FileNotFoundError:
    # Fallback por si se corre desde otro lado
    try:
        df = pd.read_csv("data/processed/datos_limpios.csv")
        print("¡Datos cargados correctamente (desde root)!")
    except:
        print("Error: No se encuentra el archivo. Asegúrate de haber corrido el preprocesamiento.")

¡Datos cargados correctamente!


## 1. La Tabla de Contingencia (Frecuencias Observadas)

El primer paso de un Chi-Cuadrado es simplemente contar. Queremos ver cuántos estudiantes caen en cada combinación de categorías.

Nuestras variables son:
-   **Rendimiento Pasado**: (Aprobé todo, Mitad, Reprobé mayoría)
-   **Intención de Abandono**: (Sí, No)

A estos conteos reales los llamamos **Frecuencias Observadas ($O$)**.

In [2]:
# Crear la tabla de contingencia
tabla_observada = pd.crosstab(df['rendimiento_semestre_pasado'], df['abandono_considerado'])

# Reordenamos para que tenga sentido lógico (de mejor a peor rendimiento)
orden = [
    'Aprobé todas o casi todas las materias que cursé',
    'Aprobé aproximadamente la mitad de las materias',
    'Reprobé más de la mitad de las materias que cursé'
]
tabla_observada = tabla_observada.reindex(orden)

print("--- Tabla de Frecuencias Observadas (O) ---")
display(tabla_observada)

--- Tabla de Frecuencias Observadas (O) ---


abandono_considerado,No,Sí
rendimiento_semestre_pasado,,
Aprobé todas o casi todas las materias que cursé,52,34
Aprobé aproximadamente la mitad de las materias,5,3
Reprobé más de la mitad de las materias que cursé,0,4


## 2. La Matemática: Frecuencias Esperadas ($E$)

Aquí es donde entra la magia (y la lógica). 

La **Hipótesis Nula ($H_0$)** dice: *"No hay relación entre el rendimiento y el abandono. Son independientes."*

Si fueran independientes, ¿cuántos estudiantes esperaríamos ver en cada celda por pura probabilidad?

La fórmula para cada celda es:

$$ E_{ij} = \frac{\text{Total Fila}_i \times \text{Total Columna}_j}{\text{Gran Total}} $$

**Ejemplo Manual:**
Vamos a calcular a mano la frecuencia esperada para la primera celda (Aprobé todo, No consideró abandonar).

In [3]:
# Cálculos manuales para entender la fórmula
total_fila_aprobe_todo = tabla_observada.loc['Aprobé todas o casi todas las materias que cursé'].sum()
total_columna_no_abandono = tabla_observada['No'].sum()
gran_total = tabla_observada.sum().sum()

esperado_manual = (total_fila_aprobe_todo * total_columna_no_abandono) / gran_total

print(f"Total estudiantes que aprobaron todo: {total_fila_aprobe_todo}")
print(f"Total estudiantes que NO pensaron abandonar: {total_columna_no_abandono}")
print(f"Gran Total de estudiantes: {gran_total}")
print(f"\nCálculo: ({total_fila_aprobe_todo} * {total_columna_no_abandono}) / {gran_total}")
print(f"Frecuencia Esperada (E) para esta celda: {esperado_manual:.2f}")

# Comparar con el valor observado real
observado_real = tabla_observada.loc['Aprobé todas o casi todas las materias que cursé', 'No']
print(f"Frecuencia Observada (O) real: {observado_real}")

diferencia = observado_real - esperado_manual
print(f"\nDiferencia (O - E): {diferencia:.2f}")
if diferencia > 0:
    print("Hay MÁS estudiantes de los esperados en este grupo (lo cual es bueno, rinden bien y no abandonan).")
else:
    print("Hay MENOS estudiantes de los esperados en este grupo.")

Total estudiantes que aprobaron todo: 86
Total estudiantes que NO pensaron abandonar: 57
Gran Total de estudiantes: 98

Cálculo: (86 * 57) / 98
Frecuencia Esperada (E) para esta celda: 50.02
Frecuencia Observada (O) real: 52

Diferencia (O - E): 1.98
Hay MÁS estudiantes de los esperados en este grupo (lo cual es bueno, rinden bien y no abandonan).


## 3. ¿Normalidad? ¡No la necesitamos!

Una duda común es: *"¿No teníamos que asumir normalidad en los datos?"*

**Respuesta Corta:** No.

**Explicación:**
La asunción de normalidad (Campana de Gauss) se usa para pruebas paramétricas como la prueba *t* o ANOVA, donde analizas **medias** de variables continuas (ej. altura, salario).

Aquí estamos analizando **conteos** (frecuencias) de variables categóricas. 

**La Asunción Real del Chi-Cuadrado:**
Lo que sí necesitamos verificar es que **tenemos suficientes datos**. Específicamente:
> **Regla:** Todas las Frecuencias Esperadas ($E$) deben ser mayores a 5 (o al menos el 80% de ellas).

Si esperamos ver menos de 5 personas en un grupo, la matemática del Chi-Cuadrado se vuelve inestable.

¡Verifiquémoslo!

In [4]:
# Ejecutamos el test completo con scipy
chi2, p, dof, expected = chi2_contingency(tabla_observada)

# Convertimos la matriz de esperados a un DataFrame para verlo bonito
tabla_esperada = pd.DataFrame(expected, index=tabla_observada.index, columns=tabla_observada.columns)

print("--- Tabla de Frecuencias Esperadas (E) ---")
display(tabla_esperada.round(2))

# Verificación de la asunción
min_esperado = expected.min()
print(f"\nLa frecuencia esperada más pequeña es: {min_esperado:.2f}")

if min_esperado >= 5:
    print("✅ CORRECTO: Todas las frecuencias esperadas son >= 5. El análisis es válido.")
else:
    print("⚠️ CUIDADO: Hay celdas con valores esperados < 5. Los resultados podrían no ser fiables.")

--- Tabla de Frecuencias Esperadas (E) ---


abandono_considerado,No,Sí
rendimiento_semestre_pasado,,
Aprobé todas o casi todas las materias que cursé,50.02,35.98
Aprobé aproximadamente la mitad de las materias,4.65,3.35
Reprobé más de la mitad de las materias que cursé,2.33,1.67



La frecuencia esperada más pequeña es: 1.67
⚠️ CUIDADO: Hay celdas con valores esperados < 5. Los resultados podrían no ser fiables.


## 4. El Estadístico Chi-Cuadrado ($\chi^2$)

Finalmente, sumamos todas esas diferencias para obtener un solo número que nos dice "qué tan lejos estamos de la independencia".

$$ \chi^2 = \sum \frac{(O - E)^2}{E} $$

1.  Restamos Observado menos Esperado ($O-E$).
2.  Elevamos al cuadrado para eliminar negativos.
3.  Dividimos por el Esperado para normalizar (no es lo mismo fallar por 5 personas en un grupo de 10 que en uno de 1000).
4.  Sumamos todo.

**Interpretación del P-valor:**
-   Si $p < 0.05$: Es muy improbable que estas diferencias sean por azar. **Hay relación.**
-   Si $p \ge 0.05$: Las diferencias podrían ser casualidad. **No hay evidencia suficiente de relación.**

In [5]:
print(f"Estadístico Chi-Cuadrado calculado: {chi2:.4f}")
print(f"P-valor: {p:.4e}") # Notación científica si es muy pequeño

alpha = 0.05
if p < alpha:
    print("\nCONCLUSIÓN FINAL: El p-valor es menor a 0.05.")
    print("Rechazamos la hipótesis nula. SÍ existe una asociación significativa entre el rendimiento académico y la intención de abandono.")
else:
    print("\nCONCLUSIÓN FINAL: El p-valor es mayor a 0.05.")
    print("No podemos rechazar la hipótesis nula. No encontramos una asociación significativa.")

Estadístico Chi-Cuadrado calculado: 5.8101
P-valor: 5.4747e-02

CONCLUSIÓN FINAL: El p-valor es mayor a 0.05.
No podemos rechazar la hipótesis nula. No encontramos una asociación significativa.
